# Setup

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import pandas as pd

from plotting import *
from prices import *
from inflation import *

# Dataset

## ML systems

In [6]:
pcd_df = pd.read_csv('data/All ML Systems - full view.csv')
pcd_df.head()

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
0,Cohere Command,Language,NaN,NaN,NaN,NaN,NaN,https://cohere.com/models/command,NaN,"World-class AI, at your command",...,Cohere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Theseus,Other,Maze solving,Claude Shannon,Historical significance,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,0.0,Mighty Mouse,...,Bell Laboratories,NaN,NaN,NaN,Theseus,NaN,NaN,NaN,NaN,NaN
2,SNARC,Other,Maze solving,Marvin Minsky,Historical significance,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,33.0,A Neural-Analogue Calculator Based upon a Prob...,...,Harvard University,NaN,NaN,NaN,SNARC,NaN,NaN,NaN,NaN,NaN
3,Genetic algorithm,NaN,NaN,NA Barricelli,Historical significance,Possibly first computer simulation of a geneti...,NaN,https://link.springer.com/article/10.1007/BF01...,266.0,Numerical testing of evolution theories,...,Institute for Advanced Study,NaN,NaN,NaN,Genetic algorithm,NaN,NaN,NaN,NaN,NaN
4,Sequence-based pattern recognition,Vision,Character recognition,O. G. Selfridge,Historical significance,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455310,290.0,Pattern recognition and modern computers,...,Massachusetts Institute of Technology (MIT),NaN,NaN,NaN,Sequence-based pattern recognition,NaN,NaN,NaN,NaN,NaN


In [7]:
# Publication date in datetime format
pcd_df.dropna(subset=['Publication date'], inplace=True)
pcd_df['Publication date'] = pd.to_datetime(pcd_df['Publication date'])

In [8]:
# Manually copied from "Training cost trends" Airtable
frontier_systems = [
    "PaLM 2",
    "GPT-4",
    "Minerva (540B)",
    "Megatron-Turing NLG 530B",
    "GPT-3 175B (davinci)",
    "Meena",
    "AlphaStar",
    "AlphaGo Zero",
    "AlphaGo Master",
    "GNMT",
    "Claude 2",
    "PaLM (540B)",
#     "ERNIE 3.0 Titan",
    "Gopher (280B)",
    "OpenAI Five",
    "T5-11B",
    "Megatron-BERT",
    "ResNeXt-101 32x48d",
    "AlphaZero",
    "Falcon 180B",
    "GPT-3.5 (text-davinci-003)",
    "Chinchilla",
    "Yuan 1.0",
    "Turing-NLG",
    "BigGAN-deep 512x512",
    "NASv3 (CIFAR-10)",
    "AlphaGo Lee",
    "AlphaGo Fan",
    "OPT-175B",
#     "AlphaCode",
    "GLaM",
    "OpenAI Five Rerun",
    "T5-3B",
    "Megatron-LM (8.3B)",
    "FTW",
    "AmoebaNet-A (F=448)",
    "OpenAI TI7 DOTA 1v1",
    "JFT",
    "Llama 2-70B",
    "LLaMA-65B",
    "LaMDA",
    "ALIGN",
    "GShard (dense)",
    "RoBERTa Large",
    "IMPALA",
]

In [9]:
frontier_pcd_df = pcd_df[pcd_df['System'].isin(frontier_systems)]
frontier_pcd_df.head()

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
265,AlphaGo Fan,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",SOTA improvement,NaN,NaN,https://www.nature.com/articles/nature24270.ep...,14389.0,Mastering the game of Go with deep neural netw...,...,Google DeepMind,NaN,NaN,NaN,AlphaGo Fan,NaN,NaN,AlphaGo Fan,NaN,NaN
275,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14389.0,Mastering the game of Go with deep neural netw...,...,DeepMind,NaN,NaN,NaN,AlphaGo Lee,NaN,NaN,AlphaGo Lee,NaN,NaN
306,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,5948.0,Google's Neural Machine Translation System: Br...,...,Google,NaN,NaN,NaN,GNMT,96.0,NaN,GNMT,NaN,NaN
317,NASv3 (CIFAR-10),Vision,NaN,"Barret Zoph, Quoc V. Le",Highly cited,NaN,NaN,https://arxiv.org/abs/1611.01578,4569.0,Neural Architecture Search with Reinforcement ...,...,Google Brain,NaN,NaN,NaN,NASv3 (CIFAR-10),800.0,NaN,NASv3 (CIFAR-10),NaN,NaN
337,AlphaGo Master,Games,Go,"D Silver, J Schrittwieser, K Simonyan, I Anton...",Highly cited,NaN,NaN,https://www.researchgate.net/publication/32047...,7831.0,Mastering the game of Go without human knowledge,...,DeepMind,NaN,NaN,NaN,AlphaGo Master,NaN,NaN,AlphaGo Master,NaN,NaN


In [10]:
assert len(frontier_pcd_df) == len(frontier_systems)

## Prices

In [11]:
price_df = pd.read_csv('data/Hardware prices.csv')
price_df.head()

,Price source,Price date,Hardware model,Manufacturer (from Hardware model),Vendor,Location,Price per chip-hour (on-demand),Price per chip-hour (1-year CUD),Price per chip-hour (3-year CUD),Price (hardware purchase)
0,https://web.archive.org/web/20181009102635/htt...,2018-10-09,Google TPU v2,Google,Google Cloud,US,$1.13,NaN,NaN,NaN
1,https://web.archive.org/web/20181011013513/htt...,2018-10-11,Google TPU v3,Google,Google Cloud,US,$2.00,NaN,NaN,NaN
2,https://web.archive.org/web/20181011013513/htt...,2018-10-11,Google TPU v3,Google,Google Cloud,Europe,$2.20,NaN,NaN,NaN
3,https://web.archive.org/web/20190701021000/htt...,2019-07-01,Google TPU v3,Google,Google Cloud,Iowa (us-central1),$2.00,$1.26,$0.90,NaN
4,https://web.archive.org/web/20190728061708/htt...,2019-07-28,Google TPU v3,Google,Google Cloud,Netherlands (europe-west4),$2.00,$1.26,$0.90,NaN


In [12]:
# Price date in datetime format
price_df.dropna(subset=['Price date'], inplace=True)
price_df['Price date'] = pd.to_datetime(price_df['Price date'])

In [13]:
pcd_hardware_model_colname = 'Name of the hardware (from Training hardware)'

## Hardware data

In [14]:
hardware_df = pd.read_csv('data/Chip dataset-Grid view.csv')
hardware_df.head()

,Name of the hardware,Manufacturer,Type,Release date,Release price (USD),FP64 (double precision) Performance (FLOP/s),FP32 (single precision) Performance (FLOP/s),FP16 (half precision) Performance (FLOP/s),Tensor Float 32 (TF32),FP16 Tensor Core,...,Foundry,Number of transistors in million,Prominent Years of usage,Google Cloud pricing ($ per hour) data from 17 dec 2022,Link to datasheet,Source for the Price,ALL ML SYSTEMS,All ML Systems copy,All ML Systems copy.1,Hardware prices
0,3dfx Spectre 1000,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3dfx Spectre 2000,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3dfx Spectre 3000,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3dfx Voodoo4 4000 AGP,Other,GPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3dfx Voodoo4 4500 AGP,Other,GPU,2000-10-13,NaN,NaN,NaN,NaN,NaN,NaN,...,TSMC,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Imputation

Training hardware: k nearest neighbors

In [15]:
# drop unneeded columns from frontier_pcd_df
irrelevant_columns = ['Notability criteria', 'Notability criteria notes', 'Link', 'Citations', 'Parameters notes',
                      'Training compute notes', 'Training dataset notes', 'Dataset size notes',
                      'Inference compute notes', 'Approach', 'Confidence', 'Last modified', 'Created By', 'Benchmark data',
                      'Exclude', 'Authors by country', 'Training cost trends', 'Abstract', 'Compute cost notes',
                      'Training time notes', 'Authors', 'Name of the hardware (from Training hardware)',
                      'Training compute cost (2020 USD)', 'Organization categorization',
                      'Training dataset', 'Inference compute (FLOP)', 'Compute sponsor categorization',
                      'Finetune compute notes']
frontier_pcd_df = frontier_pcd_df.drop(columns=irrelevant_columns)

# fill column 'Training cloud compute vendor' using org_to_cloud_vendor dictionary
org_to_cloud_vendor = {
    'Google': 'Google Cloud',
    'DeepMind': 'Google Cloud',
    'Google DeepMind': 'Google Cloud',
    'Google Brain': 'Google Cloud',
    'Microsoft': 'Microsoft Azure',
    'OpenAI': 'Microsoft Azure',
}
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Organization (from Organization)'].map(org_to_cloud_vendor)
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Training cloud compute vendor'].fillna('Amazon Web Services')

In [16]:
# Import KNNImputer from sklearn
from sklearn.impute import KNNImputer
# instantiate the imputer
imputer = KNNImputer(n_neighbors=5)

# convert datetime to float
frontier_pcd_df['Publication date'] = frontier_pcd_df['Publication date'].dt.year + (frontier_pcd_df['Publication date'].dt.month-1) / 12 + (frontier_pcd_df['Publication date'].dt.day-1) / 365

# set the System column as the index
frontier_pcd_df = frontier_pcd_df.set_index('System')

# Identify categorical columns
categorical_cols = frontier_pcd_df.select_dtypes(include=['object', 'category']).columns.tolist()

# one-hot encode all categorical columns
one_hot_pcd_df = pd.get_dummies(frontier_pcd_df, columns=categorical_cols)

# impute the missing values in Training hardware, hardware quantity, Training time (hours)
imputed = imputer.fit_transform(one_hot_pcd_df)

# convert the numpy array back to a dataframe
imputed_pcd_df = pd.DataFrame(imputed, columns=one_hot_pcd_df.columns)

# convert Training hardware back to categorical
imputed_pcd_df['Training hardware'] = ''
for col in imputed_pcd_df.columns:
    if col.startswith('Training hardware_'):
        training_hardware = col.split('Training hardware_')[1]
        imputed_pcd_df['Training hardware'] = imputed_pcd_df['Training hardware'] + pd.Series([int(_) * training_hardware for _ in imputed_pcd_df[col]])

# replace all '' with np.nan
imputed_pcd_df['Training hardware'] = imputed_pcd_df['Training hardware'].replace('', np.nan)

def impute_training_hardware(dataframe, n=5):
    # use KNeighborsClassifier to impute the missing values in Training hardware
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import LabelEncoder

    # Separate the target and features
    target_col = 'Training hardware'
    features = dataframe.drop(target_col, axis=1)
    target = dataframe[target_col]

    # Encode the target column
    label_encoder = LabelEncoder()
    target_filled = target.fillna('Unknown')  # Temporarily fill missing values
    target_encoded = label_encoder.fit_transform(target_filled)

    # Train a KNeighborsClassifier
    knc = KNeighborsClassifier(n_neighbors=n)
    knc.fit(features, target_encoded)

    # Predict the missing values
    missing_values = features[target.isna()]
    predicted = knc.predict(missing_values)

    # Decode the predictions
    predicted_labels = label_encoder.inverse_transform(predicted)

    # Replace the missing values with the predictions
    dataframe.loc[target.isna(), target_col] = predicted_labels

    # replace all 'Unknown' with np.nan
    dataframe['Training hardware'] = dataframe['Training hardware'].replace('Unknown', np.nan)

missing_values = imputed_pcd_df['Training hardware'].isna().sum()
N = 5
while missing_values > 0:
    impute_training_hardware(imputed_pcd_df, n=N)
    print(imputed_pcd_df['Training hardware'].isna().sum())
    if imputed_pcd_df['Training hardware'].isna().sum() == missing_values:
        N += 5
    else:
        missing_values = imputed_pcd_df['Training hardware'].isna().sum()

# restore the System column
imputed_pcd_df['System'] = one_hot_pcd_df.index

# set the System column as the index
imputed_pcd_df = imputed_pcd_df.set_index('System')

# insert imputed values into frontier_pcd_df
frontier_pcd_df['Training hardware'] = imputed_pcd_df['Training hardware']
frontier_pcd_df['Hardware quantity'] = imputed_pcd_df['Hardware quantity']
frontier_pcd_df['Hardware utilization'] = imputed_pcd_df['Hardware utilization']
frontier_pcd_df['Training time (hours)'] = imputed_pcd_df['Training time (hours)']
frontier_pcd_df['Training time (chip hours)'] = frontier_pcd_df['Training time (hours)'] * frontier_pcd_df['Hardware quantity']

# calculate training time (chip hours) from training time and hardware quantity
frontier_pcd_df['Training time (chip hours)'] = frontier_pcd_df['Training time (hours)'] * frontier_pcd_df['Hardware quantity']

4
2
2
0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

FLOP/second (flops) utilization: mean of known values

TODO: random sample from distribution, combined with bootstrapping

Number of chip-hours

If num_chips AND training_time_hours known: chip_hours = num_chips * training_time_hours

Else if num_chips unknown: 

Else if training_time_hours unknown: 

Else (both unknown): chip_hours = training_compute_flop / (chip_flops * flops_utilization * SECONDS_PER_HOUR)

In [17]:
chip_hours = []
for i, row in frontier_pcd_df.iterrows():
    if pd.isna(row['Hardware quantity']) or pd.isna(row['Training time (hours)']):
        # TODO impute missing values
        chip_hours.append(np.nan)
    else:
        chip_hours.append(row['Hardware quantity'] * row['Training time (hours)'])

frontier_pcd_df['Training time (chip hours)'] = chip_hours

# Price selection

1. Use a fixed mapping from Organization to cloud provider. If no mapping found, default to "Amazon Web Services".
2. If there's a match for the hardware model, use that. Else, discard the ML system from the dataset.
3. Use the price that is nearest to, but prior to, training time + 2 months before the publication date
4. If there are no prices prior to that time, use the nearest price after that time
5. If there are no prices for that hardware model and cloud provider at all, repeat steps 3 and 4 for "Microsoft Azure", then "Google Cloud" as the cloud provider.
6. If there are no prices found from step 5, discard the ML system from the dataset.

In [18]:
# Example usage
example_vendor = "Google Cloud"
example_hardware_model = "Google TPU v3"
example_date = "2019-07-15" # Example date, format should be YYYY-MM-DD

# Find the row
closest_row_df = find_closest_price_dates(example_vendor, example_hardware_model, example_date, price_df)

for i, row in closest_row_df.iterrows():
    if row['Price date'] <= pd.to_datetime(example_date):
        print(f"Price date: {row['Price date']}")
        print(f"Price: {row['Price per chip-hour (1-year CUD)']}")
        break    

Price date: 2019-07-01 00:00:00
Price: $1.26


In [19]:
org_to_cloud_vendor = {
    'google': 'Google Cloud',
    'deepmind': 'Google Cloud',
    'microsoft': 'Microsoft Azure',
    'openai': 'Microsoft Azure',
}

In [20]:
frontier_pcd_df['System'] = frontier_pcd_df.index

In [21]:
# Function to convert float year to datetime
def float_year_to_datetime(float_year):
    year = int(float_year)
    remainder = float_year - year
    days_in_year = 365 + int(pd.Timestamp(year=year, month=12, day=31).is_leap_year)
    day_of_year = int(remainder * days_in_year)
    return pd.Timestamp(year=year, month=1, day=1) + pd.to_timedelta(day_of_year, unit='D')

frontier_pcd_df['Publication date'] = frontier_pcd_df['Publication date'].apply(float_year_to_datetime)

In [22]:
pcd_hardware_model_colname = 'Training hardware'

In [23]:
price_colname = 'Price per chip-hour (1-year CUD)'
system_to_price = {}

for i, row in frontier_pcd_df.iterrows():
    price = find_price(row, price_df, hardware_df, pcd_hardware_model_colname, price_colname, org_to_cloud_vendor)
    if price is None:
        continue
    else:
        system_to_price[row['System']] = price

system_to_price

==== System: AlphaGo Fan ====
Trying Google TPU v3 at 2015-07-17 09:00:00
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 2.0

==== System: AlphaGo Lee ====
Trying Google TPU v3 at 2015-11-12 09:00:00
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 2.0

==== System: GNMT ====
Trying NVIDIA Tesla K80 at 2016-01-30 00:00:00
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 1.4

==== System: NASv3 (CIFAR-10) ====
Trying Google TPU v3 at 2016-08-21 09:00:00
Trying Google Cloud, Price per chip-hour

{'AlphaGo Fan': 2.0,
 'AlphaGo Lee': 2.0,
 'GNMT': 1.4,
 'NASv3 (CIFAR-10)': 2.0,
 'JFT': 1.4,
 'AlphaZero': 1.13,
 'AmoebaNet-A (F=448)': 0.052760065856088875,
 'IMPALA': 1.46,
 'ResNeXt-101 32x48d': 2.29,
 'FTW': 2.29,
 'BigGAN-deep 512x512': 2.0,
 'RoBERTa Large': 2.29,
 'Megatron-BERT': 0.1051277082983274,
 'Megatron-LM (8.3B)': 2.29,
 'T5-3B': 1.26,
 'T5-11B': 1.26,
 'AlphaStar': 1.26,
 'OpenAI Five Rerun': 2.29,
 'Meena': 1.26,
 'Turing-NLG': 2.29,
 'GPT-3 175B (davinci)': 2.29,
 'GShard (dense)': 1.26,
 'ALIGN': 1.26,
 'Megatron-Turing NLG 530B': 2.62,
 'Gopher (280B)': 1.26,
 'GLaM': 2.03,
 'LaMDA': 1.26,
 'Chinchilla': 2.03,
 'PaLM (540B)': 2.03,
 'OPT-175B': 3.0,
 'Minerva (540B)': 2.03,
 'GPT-3.5 (text-davinci-003)': 2.4,
 'LLaMA-65B': 2.35,
 'GPT-4': 2.4,
 'PaLM 2': 2.03,
 'Llama 2-70B': 3.0,
 'Falcon 180B': 2.4}

# Cost estimation

TODO: inflation adjustment

cost = price_per_chip_hour * chip_hours

In [24]:
def estimate_cost(row, system_to_price):
    system = row['System']
    price = system_to_price.get(system)
    if price is None:
        return None

    chip_hours = row['Training time (chip hours)']
    if np.isnan(chip_hours):
        return None

    cost = price * chip_hours

    # Check for base model
    if not pd.isna(row['Base model']):
        base_model_name = row['Base model']
        base_model = frontier_pcd_df[frontier_pcd_df['System'] == base_model_name].squeeze()
        base_cost = estimate_cost(base_model, system_to_price)
        if base_cost is None:
            return None
        else:
            cost += base_cost

    return cost
    

In [25]:
system_to_cost = {}
for i, row in frontier_pcd_df.iterrows():
    cost = estimate_cost(row, system_to_price)
    if cost is None:
        continue
    system_to_cost[row['System']] = cost

system_to_cost

{'AlphaGo Fan': 233674.55999999997,
 'AlphaGo Lee': 233674.55999999997,
 'GNMT': 580608.0,
 'NASv3 (CIFAR-10)': 600320.0,
 'JFT': 100800.0,
 'AlphaZero': 94481.7408,
 'AmoebaNet-A (F=448)': 3988.660978720319,
 'IMPALA': 146.0,
 'ResNeXt-101 32x48d': 962569.4400000001,
 'FTW': 1274258.5920000002,
 'BigGAN-deep 512x512': 24576.0,
 'RoBERTa Large': 281395.2,
 'Megatron-BERT': 74924.93821505114,
 'Megatron-LM (8.3B)': 383400.96,
 'T5-3B': 147214.9728,
 'T5-11B': 310883.328,
 'AlphaStar': 510935.04,
 'OpenAI Five Rerun': 1466772.48,
 'Meena': 928972.8,
 'Turing-NLG': 605269.3504,
 'GPT-3 175B (davinci)': 8134080.0,
 'GShard (dense)': 1300561.92,
 'ALIGN': 224050.176,
 'Megatron-Turing NLG 530B': 9037952.0,
 'Gopher (280B)': 4748083.2,
 'GLaM': 2839531.5199999996,
 'LaMDA': 1786982.4,
 'Chinchilla': 3513681.2031999994,
 'PaLM (540B)': 17062133.759999998,
 'OPT-175B': 2437632.0,
 'Minerva (540B)': 18508922.88,
 'GPT-3.5 (text-davinci-003)': 21853839.36,
 'LLaMA-65B': 2406400.0,
 'GPT-4': 1368

In [26]:
for system, cost in system_to_cost.items():
    frontier_pcd_df.loc[system, 'Cost'] = cost

# Apply inflation adjustment

In [27]:
frontier_pcd_df.columns

Index(['Domain', 'Task', 'Open-source', 'Reference', 'Publication date',
       'Organization', 'Parameters', 'Training compute (FLOP)',
       'Training dataset size (datapoints)', 'Epochs', 'Training time (hours)',
       'Training hardware', 'Country (from Organization)',
       'Organization (from Organization)', 'Base model',
       'Finetune compute (FLOP)', 'Hardware quantity', 'Hardware utilization',
       'Training cloud compute vendor', 'Training data center',
       'Training time (chip hours)', 'System', 'Cost'],
      dtype='object')

In [28]:
from_year_month = frontier_pcd_df['Publication date'].apply(str)
frontier_pcd_df['Publication date'] = from_year_month

In [29]:
frontier_pcd_df['Publication date']

System
AlphaGo Fan                   2015-10-01 00:00:00
AlphaGo Lee                   2016-01-27 00:00:00
GNMT                          2016-09-26 00:00:00
NASv3 (CIFAR-10)              2016-11-05 00:00:00
AlphaGo Master                2017-01-01 00:00:00
JFT                           2017-07-11 00:00:00
OpenAI TI7 DOTA 1v1           2017-08-11 00:00:00
AlphaGo Zero                  2017-10-18 00:00:00
AlphaZero                     2017-12-05 00:00:00
AmoebaNet-A (F=448)           2018-02-04 00:00:00
IMPALA                        2018-02-04 00:00:00
ResNeXt-101 32x48d            2018-05-03 00:00:00
FTW                           2018-07-04 00:00:00
BigGAN-deep 512x512           2018-09-28 00:00:00
RoBERTa Large                 2019-07-02 00:00:00
Megatron-BERT                 2019-09-17 00:00:00
Megatron-LM (8.3B)            2019-09-17 00:00:00
T5-3B                         2019-10-23 00:00:00
T5-11B                        2019-10-23 00:00:00
AlphaStar                     2019-10-30 00

In [ ]:
frontier_pcd_df = adjust_column_for_inflation(frontier_pcd_df, 'Cost', 'data/PCU518210518210.csv', '2023-12')

# Regression

# Plots

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for i, row in frontier_pcd_df.iterrows():
    system = row['System']
    cost = system_to_cost.get(system)
    if cost is None:
        continue
    publication_date = row['Publication date']

    fig.add_trace(go.Scatter(
        x=[publication_date],
        y=[cost],
        name=system,
        text=system,
        textposition='top center',
        line=dict(color='#034752'),
        mode='markers+text',
    ))

# log y axis
fig.update_yaxes(type="log")
# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (USD, nominal)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
# fig.update_xaxes(range=['2017-01-01', '2025-01-01'])
# fig.update_yaxes(range=[5, 8])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, 'results/', 'cost_scatter')

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for i, row in frontier_pcd_df.iterrows():
    system = row['System']
    cost = system_to_cost.get(system)
    if cost is None:
        continue
    publication_date = row['Publication date']

    fig.add_trace(go.Bar(
        x=[system],
        y=[cost],
        name=system,
        # nice blue color
        marker_color='#034752',
        # text=system,
        # textposition='auto',
    ))

# log y axis
fig.update_yaxes(type="log")
# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Cost (USD, nominal)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_yaxes(range=[0, 8])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

fig.show()


# Export data

In [ ]:
frontier_pcd_df.drop('Megatron-BERT', inplace=True)
frontier_pcd_df.drop('IMPALA', inplace=True)

In [ ]:
frontier_pcd_df.to_csv('results/price dataset.csv')